In [7]:
def gzExtract(file):
    import gzip
    import shutil
    with gzip.open(file, 'rb') as f_in:
        with open(file[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return file[:-3]

In [ ]:
def dataImport():
    import wget
    # Importing the dataset, CIC IDS 2017
    wget.download("http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/MachineLearningCSV.zip") 
    !mkdir -p "MachineLearningCSV.zip"
    !cp MachineLearningCVE.zip "/Dataset/CICIDS2017/"
    !unzip -n -d "/Dataset/CICIDS2017/" MachineLearningCSV.zip
    #-------------------------------------------------------------------#
    # Importing the dataset, CSE-CIC-IDS2018
    !kaggle datasets download -d solarmainframe/ids-intrusion-csv -p "/Dataset/CICIDS2018/" --unzip
    #-------------------------------------------------------------------#
    !wget -nc -O MachineLearningCVE.zip http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs



In [5]:
def bar_thermometer(current, total, width=80):
    percent = float(current) / total
    filled_length = int(width * percent)
    bar = '█' * filled_length + '-' * (width - filled_length)
    percent_display = f"{percent:.5%}"
    
    # Convert bytes to more readable format
    def format_size(bytes):
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if bytes < 1024:
                return f"{bytes:.2f} {unit}"
            bytes /= 1024
    
    current_size = format_size(current)
    total_size = format_size(total)
    
    sys.stdout.write(f'\r|{bar}| {percent_display} ({current_size} / {total_size})')
    sys.stdout.flush()

In [6]:
import wget
wget.download("http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0-0249.zip", bar=bar_thermometer)

|| 0.2% (16.02 MB / 7.34 GB)B)

KeyboardInterrupt: 